In [1]:
import datetime
import pandas as pd
import numpy as np
import requests
import zipfile
import io
import json

from sklearn import datasets, ensemble, model_selection
from scipy.stats import anderson_ksamp

In [2]:
content = requests.get("https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip").content
with zipfile.ZipFile(io.BytesIO(content)) as arc:
    raw_data = pd.read_csv(arc.open("hour.csv"), header=0, sep=',', parse_dates=['dteday'])

In [3]:
raw_data.index = raw_data.apply(lambda row: datetime.datetime.combine(row.dteday.date(), datetime.time(row.hr)),
                                axis=1)

In [4]:
raw_data.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
2011-01-01 00:00:00,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
2011-01-01 01:00:00,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2011-01-01 02:00:00,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
2011-01-01 03:00:00,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
2011-01-01 04:00:00,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [5]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17379 entries, 2011-01-01 00:00:00 to 2012-12-31 23:00:00
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   instant     17379 non-null  int64         
 1   dteday      17379 non-null  datetime64[ns]
 2   season      17379 non-null  int64         
 3   yr          17379 non-null  int64         
 4   mnth        17379 non-null  int64         
 5   hr          17379 non-null  int64         
 6   holiday     17379 non-null  int64         
 7   weekday     17379 non-null  int64         
 8   workingday  17379 non-null  int64         
 9   weathersit  17379 non-null  int64         
 10  temp        17379 non-null  float64       
 11  atemp       17379 non-null  float64       
 12  hum         17379 non-null  float64       
 13  windspeed   17379 non-null  float64       
 14  casual      17379 non-null  int64         
 15  registered  17379 non-null  int64  

In [6]:
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'mnth', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday']

reference = raw_data.loc['2011-01-01 00:00:00':'2011-01-28 23:00:00']
current = raw_data.loc['2011-01-29 00:00:00':'2011-02-28 23:00:00']

In [7]:
#null_hypothesis--> Two samples come from the same distribution

from scipy import stats

counter =0
for i in numerical_features:
    test, p_val= stats.ks_2samp(reference[i], current[i])

    if p_val <0.05:
        counter+=1

In [8]:
counter/7 *100

71.42857142857143

In [9]:
from scipy.stats import chi2_contingency

rejected_chi=0
def drift_chisq(sample_1, sample_2):
    return chi2_contingency([sample_1, sample_2])[1]

for i in categorical_features:
    val= drift_chisq(reference[i].value_counts(), current[i].value_counts())

    if val<0.05:
        rejected_chi+=1



In [10]:
rejected_chi

0

In [11]:
# COMPLETE MODEL BUILDING

In [12]:


X_train, X_test, y_train, y_test = model_selection.train_test_split(
    reference[numerical_features + categorical_features],
    reference['cnt'],
    test_size=0.3
)

In [13]:
regressor = ensemble.RandomForestRegressor(random_state = 0)

regressor.fit(X_train, y_train)

preds_test = regressor.predict(X_test)

In [14]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

ref_mae=mean_absolute_error(y_test,preds_test)
ref_mse=mean_squared_error(y_test,preds_test)
ref_r2 = r2_score(y_test,preds_test)


print("MAE",ref_mae)
print("MSE",ref_mse)
print("R2",ref_r2)

MAE 9.461935483870967
MSE 181.60342258064514
R2 0.9091470861230109


In [15]:
current_x=current[numerical_features + categorical_features]
current_y=current['cnt']

current_pred = regressor.predict(current_x)

In [16]:
curr_mae=mean_absolute_error(current_y,current_pred)
curr_mse=mean_squared_error(current_y,current_pred)
curr_r2 = r2_score(current_y,current_pred)


print("MAE",curr_mae)
print("MSE",curr_mse)
print("R2",curr_r2)

MAE 19.90403337969402
MSE 1040.340424200278
R2 0.7308897788499824


In [17]:
!pip install mlflow

import mlflow


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
mlflow.set_experiment("Bicycle–Sharing-New")


2025/11/07 20:53:01 INFO mlflow.tracking.fluent: Experiment with name 'Bicycle–Sharing-New' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///c:/Users/LENOVO/Desktop/MLOps/OCT-Flask-Web-App/mlruns/757011208526908350', creation_time=1762528981742, experiment_id='757011208526908350', last_update_time=1762528981742, lifecycle_stage='active', name='Bicycle–Sharing-New', tags={}>

In [19]:
with mlflow.start_run():

    mlflow.set_tag('mlflow.runName','Refrence_run')
    mlflow.log_metric("MAE",ref_mae)
    mlflow.log_metric("MSE",ref_mse)
    mlflow.log_metric("R2",ref_r2)

    mlflow.sklearn.log_model(regressor, "model")

2025/11/07 20:53:02 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

2025/11/07 20:53:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instea

In [20]:
!mlflow ui

^C


Run this command in the Terminal in the VSCode

In [24]:
# mlflow ui --port 5000

Week by week metric record

In [25]:
current = raw_data.loc['2011-01-29 00:00:00':'2011-02-28 23:00:00']

experiment_batches = [
    ('2011-01-29 00:00:00','2011-02-07 23:00:00'),
    ('2011-02-07 00:00:00','2011-02-14 23:00:00'),
    ('2011-02-15 00:00:00','2011-02-21 23:00:00'),
]

In [26]:
for i in experiment_batches:
    with mlflow.start_run():

        mlflow.set_tag('mlflow.runName','Run_'+ str(i[0])+" : "+str(i[1]))

        current_data = current.loc[i[0]: i[1]]

        current_x=current_data[numerical_features + categorical_features]
        current_y=current_data['cnt']
        current_pred = regressor.predict(current_x)

        mae= mean_absolute_error(current_y, current_pred)
        mse= mean_squared_error(current_y, current_pred)
        r2= r2_score(current_y, current_pred)

        mlflow.log_metric("MAE",mae)
        mlflow.log_metric("MSE",mse)
        mlflow.log_metric("R2",r2)


In [ ]:
# mlflow ui --port 5000

In [ ]:
#run it for the last week and you will see the degradation in MAE, MSE